In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', split='dev')
from prediction import do_prediction
result = do_prediction(swebench[0])
print(result)